In [1]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import numpy as np

df = pd.read_csv("하이닉스 power1_data.csv", thousands=',', encoding='utf-8')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Calculating correlations for Forward Selection
correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
correlations.columns = ['Feature', 'Correlation']

#Performing T-tests
t_tests = {column: ttest_ind(X[column][y == 1], X[column][y == -1], nan_policy='omit') for column in X.columns}
t_tests_sorted = sorted(t_tests.items(), key=lambda x: x[1].pvalue)
t_tests_df = pd.DataFrame(t_tests_sorted, columns=['Feature', 'T-test'])
t_tests_df['T-test'] = t_tests_df['T-test'].apply(lambda x: x.pvalue)  # Only keep p-value for simplicity

# Lasso and Ridge regression
lasso = LassoCV().fit(X, y)
ridge = RidgeCV().fit(X, y)

lasso_importance = np.abs(lasso.coef_)
ridge_importance = np.abs(ridge.coef_)

features_lasso_sorted = sorted(zip(X.columns, lasso_importance), key=lambda x: x[1], reverse=True)
features_ridge_sorted = sorted(zip(X.columns, ridge_importance), key=lambda x: x[1], reverse=True)

features_lasso_df = pd.DataFrame(features_lasso_sorted, columns=['Feature', 'Lasso Importance'])
features_ridge_df = pd.DataFrame(features_ridge_sorted, columns=['Feature', 'Ridge Importance'])

# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# Combining all the data into a single dataframe without merging by feature
final_df = pd.concat([correlations, t_tests_df.drop('Feature', axis=1), features_lasso_df.drop('Feature', axis=1), 
                      features_ridge_df.drop('Feature', axis=1), features_rf_df.drop('Feature', axis=1)], axis=1)


# 결과를 저장할 리스트 초기화
results = []

models = {
    'SVC': SVC(random_state=1),
    'LogisticRegression': LogisticRegression(random_state = 1),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state =1),
    'RandomForestClassifier': RandomForestClassifier(random_state =1),
    'GradientBoostingClassifer' : GradientBoostingClassifier(random_state=1)
}


# 피처 중요도 기반 상위 피처 선택 및 평가 반복
for i in tqdm(range(5, 16), desc='Overall Progress'):
    top_i_features = [feature for feature, importance in sorted(zip(X.columns, RandomForestClassifier(random_state=1).fit(X, y).feature_importances_), key=lambda x: x[1], reverse=True)[:i]]
    X_topi = X[top_i_features]
    
    X_train = X_topi.iloc[:1720]
    X_test = X_topi.iloc[1720:]
    
    y_train = y.iloc[:1720]
    y_test = y.iloc[1720:]
    
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
        
        # 결과를 리스트에 추가
        results.append({'Features': i, 'Model': model_name, 'Precision': precision})

# 리스트를 DataFrame으로 변환
precision_scores_df = pd.DataFrame(results)

# 결과 출력
print(precision_scores_df.pivot(index='Features', columns='Model', values='Precision'))

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.131412245069214, tolerance: 0.04858921668362156
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06537523262574041, tolerance: 0.048392929806714086
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11951387975636862, tolerance: 0.04847731434384537
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\line

Overall Progress:   0%|          | 0/11 [00:00<?, ?it/s]

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Model     DecisionTreeClassifier  GradientBoostingClassifer  \
Features                                                      
5                       0.491256                   0.519150   
6                       0.514940                   0.522631   
7                       0.521710                   0.541052   
8                       0.517741                   0.531478   
9                       0.520794                   0.549975   
10                      0.522738                   0.539414   
11                      0.475951                   0.504514   
12                      0.516532                   0.539004   
13                      0.502489                   0.516104   
14                      0.520018                   0.518932   
15                      0.532872                   0.515085   

Model     LogisticRegression  RandomForestClassifier       SVC  
Features                                                        
5                   0.517790                0.5411

In [6]:
precision_scores_df

,Features,Model,Precision
0,5,SVC,0.569621
1,5,LogisticRegression,0.517790
2,5,DecisionTreeClassifier,0.491256
3,5,RandomForestClassifier,0.541183
4,5,GradientBoostingClassifer,0.519150
5,6,SVC,0.560049
6,6,LogisticRegression,0.582621
7,6,DecisionTreeClassifier,0.514940
8,6,RandomForestClassifier,0.546946
9,6,GradientBoostingClassifer,0.522631


In [13]:
df2=precision_scores_df.loc[precision_scores_df['Model']=='LogisticRegression',]

In [14]:
df2[['Precision']]

,Precision
1,0.517790
6,0.582621
11,0.540562
16,0.523442
21,0.525016
26,0.518690
31,0.506977
36,0.508679
41,0.506573
46,0.508130
